<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>



# <center>Getting Started with Llamatrace

This guide demonstrates how to use Llamatrace, a collaboration between Arize and LlamaIndex to deliver hosted observability and evals with native support for LlamaIndex.

ℹ️ This notebook requires an OpenAI API key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [ ]:
%pip install arize-phoenix arize-phoenix-evals openai openinference-instrumentation-openai opentelemetry-sdk opentelemetry-exporter-otlp
%pip install "arize-phoenix[evals,llama-index]" "openai>=1" gcsfs nest-asyncio "openinference-instrumentation-llama-index>=2.0.0"
%pip install -U llama-index-callbacks-arize-phoenix

In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
import os
from getpass import getpass

if not (PHOENIX_API_KEY := os.getenv("PHOENIX_API_KEY")):
    PHOENIX_API_KEY = getpass("🔑 Enter your Phoenix API key: ")

os.environ["PHOENIX_API_KEY"] = PHOENIX_API_KEY

## Step 2: Setup Tracing
Let's send a trace to Hosted Phoenix! Notce the key lines below using `OTEL_EXPORTER_OTLP_HEADERS` and `app.phoenix.arize.com/v1/traces`

In [ ]:
# setup Arize Phoenix for logging/observability
import os

import llama_index.core

os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://app.phoenix.arize.com/v1/traces"
)

Setup imports

In [ ]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import nest_asyncio
import pandas as pd
from gcsfs import GCSFileSystem
from llama_index.core import (
    Settings,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from tqdm import tqdm

import phoenix as px

nest_asyncio.apply()  # needed for concurrent evals in notebook environments
pd.set_option("display.max_colwidth", 1000)

Run LlamaIndex Query on Arize Docs

In [ ]:
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
)

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
index = load_index_from_storage(
    storage_context,
)
query_engine = index.as_query_engine()

queries_url = "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(queries_url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries[:5]

for query in tqdm(queries[:5]):
    query_engine.query(query)

response = query_engine.query("What is Arize and how can it help me as an AI Engineer?")
print(response)

## Step 3: Use the Phoenix SDK
Set the `PHOENIX_CLIENT_HEADERS` and the `PHOENIX_COLLECTOR_ENDPOINT` to the appropriate values to start using hosted Phoenix!



In [ ]:
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"


px_client = px.Client()
phoenix_df = px_client.get_spans_dataframe()
print(phoenix_df.head())